In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random
import os
import cv2
import sys
from pylab import rcParams
from PIL import Image
warnings.filterwarnings('ignore')

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Input, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications import VGG16, InceptionV3, Xception, EfficientNetB3
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy) #shortens training time by 2x

In [ ]:
df_train = pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")
df_train.head()

In [ ]:
df_train["label"] = df_train["label"].astype(str) #convert to str as we want to use Categorical Cross Entropy (CCE) later on
df_train.info()

## Exploratory Data Analysis

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize=(10,8))
sns.countplot(df_train["label"], edgecolor="black", palette="mako", order=['0','1','2','3','4'])
plt.show()

Label 0: Cassava Bacterial Blight (CBB)

In [ ]:
path = "../input/cassava-leaf-disease-classification/train_images/"
df0 = df_train[df_train["label"] == "0"]
files = df0["image_id"].sample(3).tolist()

plt.figure(figsize=(15,5))
index = 0
for file in files:
    image = Image.open(path + file)
    plt.subplot(1, 3, index + 1)
    plt.imshow(image)
    plt.axis("off")
    index += 1

plt.show()

Label 1: Cassava Brown Streak Disease (CBSD)

In [ ]:
df1 = df_train[df_train["label"] == "1"]
files = df1["image_id"].sample(3).tolist()

plt.figure(figsize=(15,5))
index = 0
for file in files:
    image = Image.open(path + file)
    plt.subplot(1, 3, index + 1)
    plt.imshow(image)
    plt.axis("off")
    index += 1

plt.show()

Label 2: Cassava Green Mottle (CGM)

In [ ]:
df2 = df_train[df_train["label"] == "2"]
files = df2["image_id"].sample(3).tolist()

plt.figure(figsize=(15,5))
index = 0
for file in files:
    image = Image.open(path + file)
    plt.subplot(1, 3, index + 1)
    plt.imshow(image)
    plt.axis("off")
    index += 1

plt.show()

Label 3: Cassava Mosiac Disease (CMD)

In [ ]:
df3 = df_train[df_train["label"] == "3"]
files = df3["image_id"].sample(3).tolist()

plt.figure(figsize=(15,5))
index = 0
for file in files:
    image = Image.open(path + file)
    plt.subplot(1, 3, index + 1)
    plt.imshow(image)
    plt.axis("off")
    index += 1

plt.show()

Label 4: Healthy

In [ ]:
df4 = df_train[df_train["label"] == "4"]
files = df4["image_id"].sample(3).tolist()

plt.figure(figsize=(15,5))
index = 0
for file in files:
    image = Image.open(path + file)
    plt.subplot(1, 3, index + 1)
    plt.imshow(image)
    plt.axis("off")
    index += 1

plt.show()

## Image Augmentation (Tensorflow)

In [ ]:
batch_size=16
image_size=300

input_shape = (image_size, image_size, 3)
target_size = (image_size, image_size)

In [ ]:
img_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.experimental.preprocessing.RandomCrop(image_size, image_size),
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.25),
        tf.keras.layers.experimental.preprocessing.RandomZoom((-0.25, 0.25), (-0.25, 0.25)),
    ])

In [ ]:
path = "../input/cassava-leaf-disease-classification/train_images/"
files = df_train["image_id"].tolist()
file = random.choice(files)
image = Image.open(path + file)
plt.imshow(image)
plt.axis("off")
plt.show()

In [ ]:
image = tf.expand_dims(np.array(image), 0)

plt.figure(figsize=(14, 14))
for i in range(9):
    augmented_image = img_augmentation(image)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_image[0])
    plt.axis("off")

## Image Augmentation (Albumentations)

In [ ]:
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor

In [ ]:
from ImageDataAugmentor.image_data_augmentor import *
import albumentations as A

train_augmentations = A.Compose([
            A.RandomCrop(image_size, image_size, p=1),
            A.CoarseDropout(p=0.5),
            A.Cutout(p=0.5),
            A.Flip(p=0.5),
            A.ShiftScaleRotate(p=0.5),
            A.HueSaturationValue(p=0.5, hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2),
            A.RandomBrightnessContrast(p=0.5, brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2)),
            A.ToFloat()
            ], p=1)

val_augmentations = A.Compose([
                A.CenterCrop(image_size, image_size, p=1),
                A.ToFloat()
                ], p=1)

In [ ]:
def TFDataGenerator(train_set, val_set):
    
    train_generator = ImageDataAugmentor(augment=train_augmentations)
    val_generator = ImageDataAugmentor(augment=val_augmentations)
    
    train_datagen = train_generator.flow_from_dataframe(
                  dataframe = train_set,
                  directory='../input/cassava-leaf-disease-classification/train_images',
                  x_col='image_id',
                  y_col='label',
                  target_size=target_size,
                  batch_size=batch_size,
                  shuffle=True,
                  class_mode='categorical',
                  seed=2020)

    val_datagen = val_generator.flow_from_dataframe(
                dataframe = val_set,
                directory='../input/cassava-leaf-disease-classification/train_images',
                x_col='image_id',
                y_col='label',
                target_size=target_size,
                batch_size=batch_size,
                shuffle=False,
                class_mode='categorical',
                seed=2020)
    
    return train_datagen, val_datagen

In [ ]:
train_set = df_train.iloc[:10]
val_set = df_train.iloc[-10:]

train_datagen, val_datagen = TFDataGenerator(train_set, val_set)

In [ ]:
train_images, _ = next(train_datagen)

plt.figure(figsize=(14, 14))
for i in range(9):
    image = train_images[i]
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image)
    plt.axis("off")

plt.show()

In [ ]:
val_images, _ = next(val_datagen)

plt.figure(figsize=(14, 14))
for i in range(9):
    image = val_images[i]
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image)
    plt.axis("off")

plt.show()

## Model Building and Selection

In [ ]:
def create_cnn():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(5, activation='softmax')
    ])

    # Compile
    model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=10e-5), metrics=['accuracy'])
    return model

In [ ]:
def create_vgg16():
    
    model = Sequential()
    model.add(VGG16(input_shape = input_shape, include_top = False, weights = 'imagenet'))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(512, activation = 'relu', bias_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.001)))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation = 'softmax',dtype='float32')) #this is very important to use mixed_precision

    loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True,
                                               label_smoothing=0.2,
                                               name='categorical_crossentropy' )
    
        # Compile
    model.compile(loss=loss, optimizer=keras.optimizers.Adam(lr=1e-3), metrics=['accuracy'])
    return model

In [ ]:
def create_Inception():
    base_model = InceptionV3(include_top=False, weights="imagenet", input_shape=input_shape)

    # Rebuild top
    inputs = Input(shape=input_shape)

    model = base_model(inputs)
    pooling = GlobalAveragePooling2D()(model)
    dropout = Dropout(0.2)(pooling)

    outputs = Dense(5, activation="softmax", name="dense", dtype='float32')(dropout)

    # Compile
    inception = Model(inputs=inputs, outputs=outputs)
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
    loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.2, from_logits=True)

    inception.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    return inception

In [ ]:
def create_Xception():
    base_model = Xception(include_top=False, weights="imagenet", input_shape=input_shape)

    # Rebuild top
    inputs = Input(shape=input_shape)

    model = base_model(inputs)
    pooling = GlobalAveragePooling2D()(model)
    dropout = Dropout(0.2)(pooling)

    outputs = Dense(5, activation="softmax", name="dense", dtype='float32')(dropout)

    # Compile
    xception = Model(inputs=inputs, outputs=outputs)
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
    loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.2, from_logits=True)

    xception.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    return xception

In [ ]:
def create_EfficientNetB3():
    
    model = Sequential()
    model.add(EfficientNetB3(input_shape = input_shape, include_top = False, weights = 'imagenet'))
    model.add(GlobalAveragePooling2D())
    #model.add(Dense(64, activation = 'relu', bias_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.001)))
    model.add(Dropout(0.2))
    model.add(Dense(5, activation = 'softmax',dtype='float32'))

    loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True,
                                               label_smoothing=0.2,
                                               name='categorical_crossentropy' )
    
    # Compile
    model.compile(loss=loss, optimizer=keras.optimizers.Adam(lr=1e-3), metrics=['accuracy'])
    return model 

## Training with Stratified K-Fold Cross Validation

In [ ]:
def plot_result(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    epochs = range(len(acc))

    plt.figure(figsize=(15, 5))
    plt.plot(epochs, acc, 'b*-', label='Training accuracy')
    plt.plot(epochs, val_acc, 'r*-', label='Validation accuracy')
    plt.grid()
    plt.title('Training and validation accuracy')
    plt.ylabel("Accuracy")
    plt.xlabel("Epochs")
    plt.legend()
    plt.figure()
    plt.show()

    plt.figure(figsize=(15, 5))
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    plt.plot(epochs, loss, 'b*-', label='Training Loss')
    plt.plot(epochs, val_loss, 'r*-', label='Validation Loss')
    plt.grid()
    plt.title('Training and validation loss')
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.figure()
    plt.show()

In [ ]:
# fold_number = 0
# n_splits = 2
# epochs = 8

# tf.keras.backend.clear_session()
# sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.1, random_state=2020)
# for train_index, val_index in sss.split(df_train["image_id"], df_train["label"]):
#     train_set = df_train.loc[train_index]
#     val_set = df_train.loc[val_index]
#     train_datagen, val_datagen = TFDataGenerator(train_set, val_set)
#     model = create_cnn()
#     print("Training fold no.: " + str(fold_number+1))

#     model_name = "cnn"
#     fold_name = "fold.h5"
#     filepath = model_name + str(fold_number+1) + fold_name
#     callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=1, verbose=1, factor=0.2),
#                  EarlyStopping(monitor='val_loss', patience=3),
#                  ModelCheckpoint(filepath=filepath, monitor='val_loss', save_best_only=True)]

#     history1 = model.fit(train_datagen, epochs=epochs, validation_data=val_datagen, callbacks=callbacks)
#     fold_number += 1
#     if fold_number == n_splits:
#         print("Training finished!")

In [ ]:
# plot_result(history1)

In [ ]:
# fold_number = 0
# n_spilts = 2
# epochs = 8

# tf.keras.backend.clear_session()
# sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.1, random_state=2020)
# for train_index, val_index in sss.split(df_train["image_id"], df_train["label"]):
#     train_set = df_train.loc[train_index]
#     val_set = df_train.loc[val_index]
#     train_datagen, val_datagen = TFDataGenerator(train_set, val_set)
#     model = create_vgg16()
#     print("Training fold no.: " + str(fold_number+1))

#     model_name = "vgg16"
#     fold_name = "fold.h5"
#     filepath = model_name + str(fold_number+1) + fold_name
#     callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=1, verbose=1, factor=0.2),
#                  EarlyStopping(monitor='val_loss', patience=3),
#                  ModelCheckpoint(filepath=filepath, monitor='val_loss', save_best_only=True)]

#     history2 = model.fit(train_datagen, epochs=epochs, validation_data=val_datagen, callbacks=callbacks)
#     fold_number += 1
#     if fold_number == n_splits:
#         print("Training finished!")

In [ ]:
# plot_result(history2)

In [ ]:
# fold_number = 0
# n_splits = 2
# epochs = 8

# tf.keras.backend.clear_session()
# sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.1, random_state=2020)
# for train_index, val_index in sss.split(df_train["image_id"], df_train["label"]):
#     train_set = df_train.loc[train_index]
#     val_set = df_train.loc[val_index]
#     train_datagen, val_datagen = TFDataGenerator(train_set, val_set)
#     model = create_Inception()
#     print("Training fold no.: " + str(fold_number+1))

#     model_name = "inception "
#     fold_name = "fold.h5"
#     filepath = model_name + str(fold_number+1) + fold_name
#     callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=1, verbose=1, factor=0.2),
#                  EarlyStopping(monitor='val_loss', patience=3),
#                  ModelCheckpoint(filepath=filepath, monitor='val_loss', save_best_only=True)]

#     history3 = model.fit(train_datagen, epochs=epochs, validation_data=val_datagen, callbacks=callbacks)
#     fold_number += 1
#     if fold_number == n_splits:
#         print("Training finished!")

In [ ]:
# plot_result(history3)

In [ ]:
# fold_number = 0
# n_splits = 2
# epochs = 8

# tf.keras.backend.clear_session()
# sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.1, random_state=2020)
# for train_index, val_index in sss.split(df_train["image_id"], df_train["label"]):
#     train_set = df_train.loc[train_index]
#     val_set = df_train.loc[val_index]
#     train_datagen, val_datagen = TFDataGenerator(train_set, val_set)
#     model = create_Xception()
#     print("Training fold no.: " + str(fold_number+1))

#     model_name = "xception "
#     fold_name = "fold.h5"
#     filepath = model_name + str(fold_number+1) + fold_name
#     callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=1, verbose=1, factor=0.2),
#                  EarlyStopping(monitor='val_loss', patience=3),
#                  ModelCheckpoint(filepath=filepath, monitor='val_loss', save_best_only=True)]

#     history4 = model.fit(train_datagen, epochs=epochs, validation_data=val_datagen, callbacks=callbacks)
#     fold_number += 1
#     if fold_number == n_splits:
#         print("Training finished!")

In [ ]:
# plot_result(history4)

In [ ]:
fold_number = 0
n_splits = 2
epochs = 8

tf.keras.backend.clear_session()
sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.1, random_state=2020)
for train_index, val_index in sss.split(df_train["image_id"], df_train["label"]):
    train_set = df_train.loc[train_index]
    val_set = df_train.loc[val_index]
    train_datagen, val_datagen = TFDataGenerator(train_set, val_set)
    model = create_EfficientNetB3()
    print("Training fold no.: " + str(fold_number+1))

    model_name = "efficientnetb3"
    fold_name = "fold.h5"
    filepath = model_name + str(fold_number+1) + fold_name
    callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=1, verbose=1, factor=0.2),
                 EarlyStopping(monitor='val_loss', patience=3),
                 ModelCheckpoint(filepath=filepath, monitor='val_loss', save_best_only=True)]

    history5 = model.fit(train_datagen, epochs=epochs, validation_data=val_datagen, callbacks=callbacks)
    fold_number += 1
    if fold_number == n_splits:
        print("Training finished!")

In [ ]:
plot_result(history5)

## Model Ensembling and Inference

In [ ]:
# models = []
# for i in range(n_splits):
#     inception = load_model("./inception " + str(i+1) + "fold.h5")
#     models.append(inception)
    
# for i in range(n_splits):
#     efficientnetb3 = load_model("./efficientnetb3 " + str(i+1) + "fold.h5")
#     models.append(efficientnetb3)

In [ ]:
# ss = pd.read_csv(os.path.join('../input/cassava-leaf-disease-classification', "sample_submission.csv"))
# preds = []
# results = []

# for image_id in ss.image_id:
#     image = Image.open(os.path.join('../input/cassava-leaf-disease-classification', "test_images", image_id))
#     image = image.resize((image_size, image_size))
#     image = np.expand_dims(image, axis = 0)
#     for model in models:
#         preds.append(np.argmax(model.predict(image)))
#     res = max(set(preds), key = preds.count)
#     results.append(res)

# ss['label'] = results
# ss.to_csv('submission.csv', index = False)